# Парсинг сайта автодиспетчер

- https://www.avtodispetcher.ru/distance/

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BeautifulSoup
import fake_useragent
import time
from fake_useragent import UserAgent

# Suitable default display for floats
pd.options.display.float_format = '{:,.3f}'.format

# Отобразить все колонки
pd.set_option('max_columns', None)


In [2]:
# Открываем файл
df = pd.read_excel("2023 07 13_schema.xlsx")

In [3]:
# подготовка df
df1 = df[ ['Заявка, id'
           ,'МНО, Адрес'
           ,'МНО_Координаты (Ш,Д)'
           ,'Объект переработки, Адрес'
           ,'ОП_Координаты (Ш,Д)'
          ] ]

# переименование колонок
df1 = df1.rename(columns = {'Заявка, id':'bid_id'
                           ,'МНО, Адрес':'mno_add'
                           ,'МНО_Координаты (Ш,Д)':'mno_geo'
                           ,'Объект переработки, Адрес':'orec_add'
                           ,'ОП_Координаты (Ш,Д)':'orec_geo'
                           } )

# # Устанавливаем новый индекс
# df1 = df1.set_index('bid_id')

# Удаляем адреса без конечной геолокации
df1 = df1.dropna(subset=['orec_add'])


In [4]:
df1 = df1.query('bid_id >= 63000')
df1

,bid_id,mno_add,mno_geo,orec_add,orec_geo
849,64986,"г Тула, ул Жаворонкова, д 1","54.177406,37.602577","142400, г. Ногинск, ул. Индустриальная, д.41, ...","55.87536,38.444992"
937,64825,"Калининградская обл, г Светлый, Сосновый пер, ...","54.675037,20.123331","г Калининград, ул А.Невского, д 240","54.757254,20.552349"
944,64814,"Ставропольский край, Красногвардейский р-н, се...","45.812399,41.896662","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821"
948,64810,"Оренбургская обл, г Гай, Технологический проез...","51.441559,58.407161","Республика Башкортостан, г Уфа, Бирский тракт,...","54.8456081,56.1178162"
952,64805,"Краснодарский край, Каневской р-н, ст-ца Челба...","45.972557,39.369893","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821"


In [23]:
# Скрейпер дистанции
def get_distance(from_address, to_address):
#     user_agent = UserAgent()  # Инициализируем объект UserAgent для генерации fake user-agent
#     headers = {'User-Agent': user_agent.random}  # Генерируем случайный fake user-agent
    url = f'https://www.avtodispetcher.ru/distance/?from={from_address}&to={to_address}'
    resp = requests.get(url)  #, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
    distance = soup.find('span', {'id': 'totalDistance'}).text
    time.sleep(5)  # Добавляем паузу между запросами для эмуляции человеческого поведения
    print('*')
    return distance

# Добавить колонку
df1['distance'] = df1.apply(lambda x: get_distance(x['mno_geo'], x['orec_geo']), axis=1)
df1['distance'] = df1['distance'].astype('int64')
df1

*
*
*
*
*


,bid_id,mno_add,mno_geo,orec_add,orec_geo,distance
849,64986,"г Тула, ул Жаворонкова, д 1","54.177406,37.602577","142400, г. Ногинск, ул. Индустриальная, д.41, ...","55.87536,38.444992",236
937,64825,"Калининградская обл, г Светлый, Сосновый пер, ...","54.675037,20.123331","г Калининград, ул А.Невского, д 240","54.757254,20.552349",33
944,64814,"Ставропольский край, Красногвардейский р-н, се...","45.812399,41.896662","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821",295
948,64810,"Оренбургская обл, г Гай, Технологический проез...","51.441559,58.407161","Республика Башкортостан, г Уфа, Бирский тракт,...","54.8456081,56.1178162",607
952,64805,"Краснодарский край, Каневской р-н, ст-ца Челба...","45.972557,39.369893","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821",277


In [24]:
df1.dtypes

bid_id       int64
mno_add     object
mno_geo     object
orec_add    object
orec_geo    object
distance     int64
dtype: object

In [25]:
# Сохраняем файл
df1.to_excel(f"{pd.Timestamp.now().strftime('%Y %m %d')}_Дистанция от МНО до ОП.xlsx")